## Network Setup

1. Download Multimodal network dataset from [https://gis.utah.gov/data/transportation/street-network-analysis/]
2. Download Wasatch Front TAZs from WFRC [https://data.wfrc.org/datasets/access-to-opportunities-work-related-taz-based/] (https://data.wfrc.org/datasets/access-to-opportunities-work-related-taz-based/)
2. Copy the Multimodal network dataset and delete the layers that will be replaced with clipped versions
3. Clip the NetworkDataset>BikePedAuto and NetworkDataset>Network_Dataset_ND_Junctions layers using 15 mile buffer around the Wasatch Front TAZ layer
4. Create and fill RoadClass attribute to a usable hierarchy value
5. Join the TDM data for speeds


** Note: sometimes sequential execution of the cells below will produce a "Cannot acquire a lock" error. If this happens, just re-run the cell. Esri weirdness! **


## TAZ Centroids

Calculate a centroid point for each TAZ from the TAZ shapefile. If you have an "Advanced" license of ArcGIS Pro you can do this with the "Feature to Point" tool. Otherwise, just fire up QGIS, open the TAZ layer, Go to the menu -> Vector -> Geometry tools -> polygon centroid and create a centroid point layer.

# Network Setup

Vertical Elevation - the current dataset does not contain the necessary attributes to establish vertical connectivity. Elevation is not used.

Hierarchy - oddly, adding a heirarchy for routing seems to worsen performance. Hierarchy is not used.

In [ ]:
from arcgis.gis import GIS
import arcpy
import os


# Update this
base_path = r"c:\wfrc\ato"

source_mm_network_dataset = r"c:\wfrc\ato\shp\MM_NetworkDataset_08052021.gdb\NetworkDataset"
tdm_shp_path = r"c:\wfrc\ato\shp\Master_Segs_withFactors_20200504_BY2019SegSummary\Master_Segs_withFactors_20200504_BY2019SegSummary.shp"

base_gdb = os.path.join(base_path, "ato.gdb")
if not os.path.isdir(base_gdb):
    arcpy.management.CreateFileGDB(base_path, "ato")

arcpy.env.workspace = base_gdb

In [ ]:
# copy TAZ polygons
arcpy.management.Copy(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\ATO"), 
    os.path.join(base_gdb, "taz_ato")
)

In [ ]:
# copy TAZ centroids
arcpy.management.Copy(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids"), 
    os.path.join(base_gdb, "taz_centroids")
)

In [ ]:
# copy Draper TAZ centroids for testing
arcpy.management.Copy(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_draper"), 
    os.path.join(base_gdb, "taz_centroids_draper")
)

In [ ]:
# Copy NetworkDataset to our working GDB
arcpy.management.Copy(
    source_mm_network_dataset, 
    os.path.join(base_gdb, "NetworkDataset")
)

In [ ]:
# delete existing network
arcpy.management.Delete(os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND"))

In [ ]:
# add hierarchy to clipped BikePedAuto
arcpy.AddField_management(r"NetworkDataset\BikePedAuto", "hierarchy", "SHORT")

In [ ]:
# if this fails due to a "lock" error try running again
# Set local variables
inTable = r"NetworkDataset\BikePedAuto"
fieldName = "hierarchy"
# note, this field was previously RoadClass and is now CartoCode
expression = "getClass(!CartoCode!)"

codeblock = """
hierarchy = {
	'1 Interstates': 1,
	'2 US Highways, Separated': 1,
	'3 - Paved Shared Use': 3,
	'3 US Highways, Unseparated': 1,
	'4 Major State Highways, Separated': 1,
	'5 Major State Highways, Unseparated': 1,
	'6 Other State Highways (Institutional)': 1,
	'7 Ramps, Collectors': 2,
	'8 - Bridge, Tunnel': 2,
	'8 Major Local Roads, Paved': 2,
	'9 - Link': 3,
	'9 Major Local Roads, Not Paved': 3,
	'10 Other Federal Aid Eligible Local Roads': 2,
	'11 Other Local, Neighborhood, Rural Roads': 3,
	'12 Other': 3,
	'13 Non-road feature': None,
	'14 Driveway': None,
	'15 Proposed': None,
	'17 Service Access Roads': None,
	'99 - UDOT FAE Calibration (Non-Road Feature)': None
}
def getClass(rc):
    return 3 if rc is None else hierarchy[rc]
"""

# Execute CalculateField 
arcpy.CalculateField_management(inTable, fieldName, expression, "PYTHON3", codeblock)

## manual intervention required

* I'm not sure why, but it is now necessary to remove and add the BikePedAuto layer from the map before running the cells below

In [ ]:
# Select non-road features
arcpy.management.SelectLayerByAttribute(
    "BikePedAuto", 
    "NEW_SELECTION", 
    "hierarchy IS NULL"
)

In [ ]:
# Create a 5 mile buffer
arcpy.analysis.Buffer(
    in_features = os.path.join(base_gdb, "taz_ato"),
    out_feature_class = os.path.join(base_gdb, "taz_buffer"),
    buffer_distance_or_field = "5 Miles",
    dissolve_option = "ALL"
)

In [ ]:
# Select all segments outside buffer
arcpy.management.SelectLayerByLocation(
    in_layer = "BikePedAuto",
    overlap_type = 'INTERSECT', 
    select_features = os.path.join(base_gdb, "taz_buffer"),
    selection_type = 'ADD_TO_SELECTION',
    invert_spatial_relationship = True
)

In [ ]:
arcpy.management.DeleteFeatures("BikePedAuto")
# sometimes this fails - removing from map and adding again seems to help

In [ ]:
arcpy.management.Delete(os.path.join(base_gdb, "taz_buffer"))

# Integrate

To fix some issues where lines don't connect, highlight all features where AutoRoute == Y and then use the Pairwise Integrate tool on BikePedAuto

In [ ]:
# Select non-road features
arcpy.management.SelectLayerByAttribute(
    "BikePedAuto", 
    "NEW_SELECTION", 
    "AutoNetwork = 'Y'"
)
# pairwise integrate
arcpy.analysis.PairwiseIntegrate("BikePedAuto", None)

In [ ]:
arcpy.management.SelectLayerByAttribute(
    "BikePedAuto", 
    "CLEAR_SELECTION",
)

### Join in Speed Data
WFRC has provided free flow and peak hour speed data from their TDM. We will join this to the network dataset for later use when roughly estimating the short-term improvement in automobile mobility associated with roadway widening.

In [ ]:
arcpy.conversion.FeatureClassToFeatureClass(
    tdm_shp_path,
    base_gdb,
    "TDM"
)

In [ ]:
# Create and calculate PK_SPD field based on minimum (peak time) speed
arcpy.management.CalculateField(
    "TDM", 
    "PK_SPD", 
    "min(!AM_SPD!,!MD_SPD!,!PM_SPD!,!EV_SPD!)", 
    "PYTHON3", '', "DOUBLE"
)

In [ ]:
# Spatially join TDM travel times to MM network:
arcpy.analysis.SpatialJoin(
    "BikePedAuto", 
    "TDM", 
    os.path.join(base_gdb, "TDM_SpatialJoin"), 
    "JOIN_ONE_TO_ONE", 
    "KEEP_ALL", 
    None, 
    "SHARE_A_LINE_SEGMENT_WITH"
)

In [ ]:
# Calculate drive times based on peak hour TDM speeds
codeblock = """
def getSpeed(spd, len):
    return None if spd is None else float(60*len/spd)
"""

arcpy.CalculateField_management(
    "TDM_SpatialJoin", 
    "DriveTime_Peak", 
    "getSpeed(!PK_SPD!, !Length_Miles!)", 
    "PYTHON3", codeblock, "DOUBLE"
)
arcpy.CalculateField_management(
    "TDM_SpatialJoin", 
    "DriveTime_FF", 
    "getSpeed(!FF_SPD!, !Length_Miles!)", 
    "PYTHON3", codeblock, "DOUBLE"
)

In [ ]:
# Update Drive Time based on TDM peak value when set/exists
expression = "!DriveTime! if !DriveTime_Peak! is None else !DriveTime_Peak!"

arcpy.CalculateField_management(
    "TDM_SpatialJoin", 
    "DriveTime", 
    expression, 
    "PYTHON3", None, "DOUBLE")

In [ ]:
# create network dataset from template
arcpy.na.CreateNetworkDatasetFromTemplate(
    os.path.join(base_path, "ND_Template_Aug21.xml"),                           
    os.path.join(base_gdb, "NetworkDataset")
)

# Modify Template

If any further changes need to be made to the NetworkDataset configuration, make them before running BuildNetwork below

In [ ]:
# finally, build the dataset
arcpy.na.BuildNetwork(os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND"))